In [1]:
import numpy as np, matplotlib as mpl, matplotlib.pyplot as plt, pandas as pd
import seaborn as sns, math, os
import statsmodels.api as sm
df_brent = pd.read_csv('brent_price.csv')
df_RUB = pd.read_csv('RUB=X.csv')
df_brent['Date'] = pd.to_datetime(df_brent['Date'])
df_RUB['Date'] = pd.to_datetime(df_RUB['Date'])

In [2]:
# Задача 1
# Проверить на стационарность ряд цены на нефть, для этого найдем Критерий Дики-Фуллера
brent = sm.tsa.stattools.adfuller(df_brent['Close*'])
print(f'Критерий Дики-Фуллера: p-value={brent}')

Критерий Дики-Фуллера: p-value=(-2.286746902649301, 0.17627799214544843, 4, 248, {'1%': -3.4569962781990573, '5%': -2.8732659015936024, '10%': -2.573018897632674}, 903.497658902387)


Вывод: Ряд цен на нефть не стационарен даже при 10% уровне значимости.

In [3]:
# Задача 2 Проверить на стационарность ряд из лог-приростов цены на нефть
# для этого прологарифмируем цену
df_brent['log2'] = np.log2(df_brent['Close*'])
df_brent_log = df_brent['log2']
# найдем Критерий Дики-Фуллера
brent_log = sm.tsa.stattools.adfuller(df_brent_log)
print(f'Критерий Дики-Фуллера: p-value={brent_log}')

Критерий Дики-Фуллера: p-value=(-2.6802466349593304, 0.07751928119837088, 0, 252, {'1%': -3.4565688966099373, '5%': -2.8730786194395455, '10%': -2.5729189953388762}, -1002.6902895666972)


Вывод: Ряд из лог-приростов цены на нефть также не стационарен даже при 10% уровне значимости.

In [6]:
# Задача 3. Построить линейную регрессию курса доллара США к рублю к 
# цене на нефть и проверить значимость параметров.
df_RUB = pd.read_csv('RUB=X.csv')
df_RUB['Date'] = pd.to_datetime(df_RUB['Date'])
# выбераем поля и объеденим датафреймы
df_brent = df_brent[['Date', 'Close*']]
df_RUB = df_RUB[['Date', 'Close']]
df_RUB_brent = df_brent.merge(df_RUB, on=["Date"])

In [8]:
# создаю и обучаю модель
X = df_RUB_brent[['Close']]
y = df_RUB_brent[['Close*']]
X = sm.add_constant(X)
model_brent_log = sm.OLS(y, X).fit()
print(model_brent_log.summary())

                            OLS Regression Results                            
Dep. Variable:                 Close*   R-squared:                       0.152
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     39.68
Date:                Fri, 08 Dec 2023   Prob (F-statistic):           1.60e-09
Time:                        00:28:54   Log-Likelihood:                -672.04
No. Observations:                 223   AIC:                             1348.
Df Residuals:                     221   BIC:                             1355.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         67.3797      2.394     28.142      0.0

Коффициент Durbin-Watson = 0.175, 

In [13]:
x, y = [df_RUB_brent_log['Close'], df_RUB_brent_log['Close*']]
res = stats.pearsonr(x, y)
res

PearsonRResult(statistic=0.3901334745607005, pvalue=1.5994870580440909e-09)

Коффициент Пирсона=0.3901334745607005 говорит об умеренной положительной связи

In [9]:
# Задача 4. Построить линейную регрессию логарифма курса доллара США к рублю к
# логарифму цены на нефть и проверить значимость параметров.
df_RUB['logR'] = np.log2(df_RUB['Close'])
df_brent['log2'] = np.log2(df_brent['Close*'])
df_brent_log = df_brent['log2']
df_RUB_brent_log = df_brent.merge(df_RUB, on=["Date"])
# создаю и обучаю модель
X = df_RUB_brent_log[['logR']]
y = df_RUB_brent_log[['log2']]
X = sm.add_constant(X)
model_brent_log = sm.OLS(y, X).fit()
print(model_brent_log.summary())

                            OLS Regression Results                            
Dep. Variable:                   log2   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     20.72
Date:                Fri, 08 Dec 2023   Prob (F-statistic):           8.77e-06
Time:                        00:31:12   Log-Likelihood:                 222.37
No. Observations:                 223   AIC:                            -440.7
Df Residuals:                     221   BIC:                            -433.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.8283      0.117     49.827      0.0

Durbin-Watson = 0.216

In [12]:
x, y = [df_RUB_brent_log['logR'], df_RUB_brent_log['log2']]
res = stats.pearsonr(x, y)
res

PearsonRResult(statistic=0.292785261783931, pvalue=8.770840440287372e-06)

Коффициент Пирсона = 0.292785261783931 говорит о слабоположительно/умеренной связи

In [10]:
# Задача 5. Исследовать автокорреляцию лог-приростов цены на нефть.
X = df_RUB_brent_log[['Close*']]
y = df_RUB_brent_log[['log2']]
X = sm.add_constant(X)
model_brent_log = sm.OLS(y, X).fit()
print(model_brent_log.summary())

                            OLS Regression Results                            
Dep. Variable:                   log2   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.410e+05
Date:                Fri, 08 Dec 2023   Prob (F-statistic):          5.41e-312
Time:                        00:31:55   Log-Likelihood:                 932.63
No. Observations:                 223   AIC:                            -1861.
Df Residuals:                     221   BIC:                            -1854.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9250      0.004   1285.825      0.0

Durbin-Watson = 0.282

In [11]:
from scipy import stats
x, y = [df_RUB_brent_log['Close*'], df_RUB_brent_log['log2']]
res = stats.pearsonr(x, y)
res

PearsonRResult(statistic=0.9992170290713979, pvalue=5.413851086797e-312)

Коффициент Пирсона = 0.9992170290713979 означает сильную положительную связь